# Lab Report 4: Using APIs in Python
## Name: Afnan Alabdulwahab
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Problem 0
Import the following libraries:

In [39]:
import numpy as np
import pandas as pd
import requests
import json
import os
import dotenv
import sys
sys.tracebacklimit = 0 # turn off the error tracebacks

## Problem 1

To obtain access to the Genius API, I followed several key steps as outlined in the documentation. First, I navigated to the API Client Management Page where I initiated the creation of a new API client. This required me to sign up for a free Genius account. After signing up and logging in, I proceeded to the API registration page. Here, I needed to provide some basic information about the API client I was creating.

I followed suit and named my client "Collab" and used the URL for the UVA Collab main page (https://collab.its.virginia.edu/portal?containerLogin=true) for both the Application Website URL and the Redirect URL. After submitting the form with these details, Genius provided me with the necessary credentials: the **client_id**, **client_secret**, and **client_access_token**.

## Problem 2
## Loading Environment Variables with dotenv
`dotenv.load_dotenv()` loads environment variables from a `.env` file into the environment. The `.env` file contains key-value pairs for the Genius API credentials I obtained in the previous step
The next lines of code access the environment variables id, secret, and accesstoken and store their values in the respective Python variables **genius_id**, **genius_secret**, and **genius_accesstoken**. The `os.getenv()` function retrieves the value of the specified environment variable, allowing us to use these credentials in our API requests without hard-coding them into the script.

In [40]:
dotenv.load_dotenv()

genius_id = os.getenv('id')
genius_secret = os.getenv('secret')
genius_accesstoken = os.getenv('accesstoken')

## Problem 3

First, I get the user agent, using 'https://httpbin.org', to include in the `headers` parameter for HTTP and API requests.

In [41]:
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']
headers = { 'User-agent': useragent }

## Setting Up API Root and Endpoint:
* root = 'https://api.genius.com': This variable defines the base URL (root) for accessing the Genius API.
* endpoint = '/search': This variable specifies the specific endpoint (/search) within the Genius API. The /search endpoint is used for searching across Genius's database.

## Making the API Request:
I use the `requests` library to send a request to the Genius API
* root + endpoint: Concatenates the base URL (root) and the endpoint to form the complete URL (https://api.genius.com/search) for the API request.
* headers=headers: headers contains user agent info
* params=params: Sends the parameters defined in the params dictionary (access_token and q (search term, Bob Dylan, in this case)) as part of the request.

In [42]:
root = 'https://api.genius.com'
endpoint = '/search'
params = {
    'access_token': genius_accesstoken,
    'q': 'Bob Dylan'
}
r = requests.get(root + endpoint, headers=headers, params=params)
r

<Response [200]>

To extract the JSON data, I used the `.text` attribute and the json.loads() function. Then, I navigated the JSON structure to find the portion that shows the API endpoint for Bob Dylan.

In [43]:
bobdylan_json = json.loads(r.text)
bobdylan_json['response']['hits'][0]['result']['primary_artist']

{'api_path': '/artists/181',
 'header_image_url': 'https://images.genius.com/e87fb11dd7f33cc7fd1a8915ad36f7c9.1000x667x1.png',
 'id': 181,
 'image_url': 'https://images.genius.com/571780cc8959528e8166e5894ab13d6f.1000x1000x1.jpg',
 'is_meme_verified': False,
 'is_verified': False,
 'name': 'Bob Dylan',
 'url': 'https://genius.com/artists/Bob-dylan'}

Here, I extracted Bob Dylan's ID and API endpoint path from the JSON data.

In [47]:
bob_dylan_id = bobdylan_json['response']['hits'][0]['result']\
['primary_artist']['id']
bob_dylan_api_path = bobdylan_json['response']['hits'][0]['result']\
['primary_artist']['api_path']
print(f"Bob Dylan's ID: {bob_dylan_id}")
print(f"Bob Dylan's API Endpoint Path: {bob_dylan_api_path}")

Bob Dylan's ID: 181
Bob Dylan's API Endpoint Path: /artists/181


## Problem 4

<img src="img.jpg" width="600">

Using Bob Dylan's API endpoint path with `/songs` to request the 20 most popular Bob Dylan songs. I used to documentation to find the parameters specifications for the `/songs` request.

## Defining Parameters for the Request:
* 'access_token': This parameter is set to genius_accesstoken, holding the access token required for authenticating and accessing the Genius API.
* 'sort': This parameter specifies how the results should be sorted. In this case, it is set to 'popularity'.
* 'per_page': This parameter specifies the number of results per page to be returned by the API. Here, it is set to 20.

In [45]:
endpoint = bob_dylan_api_path + '/songs'
params = {
    'access_token': genius_accesstoken,
    'sort': 'popularity',
    'per_page': 20
         }
r = requests.get(root + endpoint, headers=headers, params=params)
r

<Response [200]>

Used `json.loads()` on the `.text` attribute of the output `r` to register the data in Python's memory.

In [29]:
bdsongs_json = json.loads(r.text)

Then, used `pd.json_normalize()` on the list `bdsongs_json` to store every feature in the data in a separate column and save the data as a `pandas` dataframe.

In [48]:
bdsongs_df = pd.json_normalize(bdsongs_json, record_path = \
                               ['response', 'songs'])
bdsongs_df

,annotation_count,api_path,artist_names,full_title,header_image_thumbnail_url,header_image_url,id,lyrics_owner_id,lyrics_state,path,...,stats.pageviews,primary_artist.api_path,primary_artist.header_image_url,primary_artist.id,primary_artist.image_url,primary_artist.is_meme_verified,primary_artist.is_verified,primary_artist.name,primary_artist.url,stats.concurrents
0,15,/songs/96286,USA For Africa,We Are the World by USA For Africa,https://images.genius.com/c21fb053cb43a6a35e31...,https://images.genius.com/c21fb053cb43a6a35e31...,96286,4733728,complete,/Usa-for-africa-we-are-the-world-lyrics,...,650054,/artists/370890,https://images.genius.com/3fb2d9f68c911b547339...,370890,https://images.genius.com/3fb2d9f68c911b547339...,False,False,USA For Africa,https://genius.com/artists/Usa-for-africa,NaN
1,12,/songs/79424,Bob Dylan,Blowin' in the Wind by Bob Dylan,https://images.genius.com/84e1705bc64495197216...,https://images.genius.com/84e1705bc64495197216...,79424,73267,complete,/Bob-dylan-blowin-in-the-wind-lyrics,...,602584,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN
2,1,/songs/68146,Adele,Make You Feel My Love by Adele,https://images.genius.com/becead7e44b8202463f0...,https://images.genius.com/becead7e44b8202463f0...,68146,82481,complete,/Adele-make-you-feel-my-love-lyrics,...,587443,/artists/2300,https://images.genius.com/87aa5d8c32965a10e0e7...,2300,https://images.genius.com/8a23ab928ccfce13accf...,False,False,Adele,https://genius.com/artists/Adele,NaN
3,18,/songs/62605,Bob Dylan,The Times They Are A-Changin' by Bob Dylan,https://images.genius.com/e00b4dcec135cb5ab3ee...,https://images.genius.com/e00b4dcec135cb5ab3ee...,62605,49202,complete,/Bob-dylan-the-times-they-are-a-changin-lyrics,...,524320,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN
4,128,/songs/5393247,Bob Dylan,Murder Most Foul by Bob Dylan,https://images.genius.com/7fbaf838ffdf99315408...,https://images.genius.com/7fbaf838ffdf99315408...,5393247,3360167,complete,/Bob-dylan-murder-most-foul-lyrics,...,504643,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN
5,7,/songs/103982,Guns N' Roses (Ft. ​The Waters),Knockin’ on Heaven’s Door by Guns N' Roses (Ft...,https://images.genius.com/cb2614550098bf753ab3...,https://images.genius.com/cb2614550098bf753ab3...,103982,168784,complete,/Guns-n-roses-knockin-on-heavens-door-lyrics,...,499002,/artists/637,https://images.genius.com/83c4b54659d674bd3778...,637,https://images.genius.com/53ddc7640957b4279d9c...,False,False,Guns N’ Roses,https://genius.com/artists/Guns-n-roses,NaN
6,15,/songs/98664,Bob Dylan,All Along the Watchtower by Bob Dylan,https://images.genius.com/f727dfc7b28942c59cd6...,https://images.genius.com/f727dfc7b28942c59cd6...,98664,779,complete,/Bob-dylan-all-along-the-watchtower-lyrics,...,494230,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN
7,28,/songs/54784,Bob Dylan,Like a Rolling Stone by Bob Dylan,https://images.genius.com/5de9704b9bd8773ddbcb...,https://images.genius.com/5de9704b9bd8773ddbcb...,54784,7,complete,/Bob-dylan-like-a-rolling-stone-lyrics,...,461350,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN
8,10,/songs/84846,Bob Dylan,"Don't Think Twice, It's All Right by Bob Dylan",https://images.genius.com/84e1705bc64495197216...,https://images.genius.com/84e1705bc64495197216...,84846,125524,complete,/Bob-dylan-dont-think-twice-its-all-right-lyrics,...,4

## Problem 5
## Using the `lyricsgenius` library to download and display the lyrics to "Tangled Up in Blue" by Bob Dylan

Importing the package and initiating Genius:

In [32]:
import lyricsgenius
genius = lyricsgenius.Genius(genius_accesstoken)

Based on the usage instructions in the GitHub page, I can used the following syntax to search for single song for an artist: `song = genius.search_song("To You", artist.name)`:

In [37]:
song = genius.search_song("Tangled Up in Blue", "Bob Dylan")
print(song.lyrics)

Searching for "Tangled Up in Blue" by Bob Dylan...
Done.
96 ContributorsTangled Up in Blue Lyrics[Verse 1]
Early one morning the sun was shining
I was laying in bed
Wondering if she'd changed at all
If her hair was still red
Her folks they said our lives together
Sure was going to be rough
They never did like Mama's homemade dress
Papa's bankbook wasn't big enough
And I was standing on the side of the road
Rain falling on my shoes
Heading out for the East Coast
Lord knows I've paid some dues
Getting through
Tangled up in blue

[Verse 2]
She was married when we first met
Soon to be divorced
I helped her out of a jam, I guess
But I used a little too much force
We drove that car as far as we could
Abandoned it out west
Split up on a dark sad night
Both agreeing it was best
She turned around to look at me
As I was walking away
I heard her say over my shoulder
"We'll meet again someday
On the avenue"
Tangled up in blue
See Bob Dylan LiveGet tickets as low as $24You might also like[Verse 3]
